# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [7]:
# import libraries
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load
from sqlalchemy import create_engine
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import precision_recall_fscore_support as score
import re

import nltk
from nltk import pos_tag
from nltk.corpus import wordnet, stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer 

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer



# Download nltk ressources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [15]:
# load data from database
def load_data():
    engine = create_engine('sqlite:///data.db')
    df = pd.read_sql('select * from emergency_messages', engine)
    X = df['message']
    Y = df.drop(['message', 'genre'], axis = 1)
    return X, Y

# Load the data
X, Y = load_data()

### 2. Write a tokenization function to process your text data

In [3]:
# Contractions dictionary to replace in the text.
contract_dict = {"ain't": "am not", "aren't": "am not", "can't": "cannot", "can't've": "cannot have",
    "'cause": "because", "could've": "could have", "couldn't": "could not", "couldn't've": "could not have",
    "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hadn't've": "had not have",
    "hasn't": "has not", "haven't": "have not", "he'd": "he would",  "he'd've": "he would have", "he'll": "he will",
    "he'll've": "he will have", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will",
    "how's": "how is", "i'd": "I would", "i'd've": "I would have", "i'll": "I will", "i'll've": "I will have",
    "i'm": "i am", "i've": "I have", "isn't": "is not", "it'd": "it would","it'd've": "it would have",
    "it'll": "t will", "it'll've": "it will have", "it's": "t is", "let's": "let us", "ma'am": "madam",
    "mayn't": "may not", "might've": "might have", "mightn't": "might not", "mightn't've": "might not have",
    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not",
    "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have",
    "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would",
    "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
    "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
    "so's": "so is", "that'd": "that would", "that'd've": "that would have", "that's": "that is",
    "there'd": "there would", "there'd've": "there would have", "there's": "there is", "they'd": "they would",
    "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are",
    "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have",
    "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not",
    "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is",
    "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
    "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
    "why's": "why is", "why've": "why have","will've": "will have", "won't": "will not","won't've": "will not have",
    "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
    "y'all'd": "you all would", "y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have",
    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
    "you're": "you are", "you've": "you have"}

In [4]:


def tokenize(text):    
    # Finding and replacing urls
    text = re.sub('(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', 
                  'placeholder', text)

    # Replace contractions
    for key in contract_dict: 
        text = text.replace(key.lower(), contract_dict[key])
    
    # Remove special characters
    text = re.sub('[^0-9a-zA-Z]+', ' ', text)
    
    # Lower
    text = text.lower()
    
    # stop words
    stop_words = stopwords.words("english")
    
    # Tokens
    tokens = word_tokenize(text)
    
    # Part of speech
    pos_tokens = pos_tag(tokens)
    
    # Lemmatize
    clean_tokens = [WordNetLemmatizer().lemmatize(token[0], wordnet_pos(token[1])) 
                    for token in pos_tokens if token[0] not in stop_words]
    
    return clean_tokens

In [5]:
# Class to select the column with the messages only
class SelectText(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X['message'].values

# Class to categorical
class CategoricalData(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # pre-defining dataframe to be able to predict for one type
        type_cat = pd.api.types.CategoricalDtype(categories=['direct', 'news', 'social'], ordered = False)
        cat = pd.Series(X['genre'].values, dtype = type_cat)
        return pd.get_dummies(cat)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
def create_pipeline_cat():
    '''
    Pipeline if the type of text is available.
    '''
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('text', SelectText()),
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('categorical_data', CategoricalData())
        ])),

        ('model', MultiOutputClassifier(RandomForestClassifier()))
    ])
    return pipeline

def create_pipeline():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('model', MultiOutputClassifier(RandomForestClassifier()))
    ])
    return pipeline

pipeline = create_pipeline()

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
y_pred = pipeline.predict(X_test)

In [18]:
def show_performance(y_true, y_pred):
    # classification report
    fscores = []
    recalls = []
    precisions = []
    accuracies = []
    
    # Transforming preds to dataframe
    
    y_pred = pd.DataFrame(y_pred, columns = Y.columns)
    # Report
    for column in y_pred.columns:
        report = classification_report(y_test[column],y_pred[column])
#         print(report)
        precision,recall,fscore,_=score(y_test[column],y_pred[column], average = 'macro')
        accuracies.append(accuracy_score(y_test[column], y_pred[column]))
        precisions.append(precision)
        recalls.append(recall)
        fscores.append(fscore)


    # Printing metrics
    print(f'Overall Accuracy: {round(np.mean(accuracies),2)}')
    print(f'Overall f-score: {round(np.mean(fscores),2)}')
    print(f'Overall recall: {round(np.mean(recalls),2)}')
    print(f'Overall precision: {round(np.mean(precisions),2)}')

show_performance(y_test, y_pred)

Overall Accuracy: 0.94
Overall f-score: 0.6
Overall recall: 0.58
Overall precision: 0.75


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
# reducing parameters for speed
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
#     'vect__max_df': [0.75, 1.0],
#     'vect__min_df': [0.01, 0.02],
#     'tfidf__use_idf': [True, False],
#     'model__estimator__n_estimators': [50, 100]
#     'model__estimator__min_samples_split': [3, 4],
}

# Grid search
cv = GridSearchCV(pipeline, param_grid=parameters, verbose = 2, n_jobs = -1)

# fit
cv.fit(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] vect__ngram_range=(1, 1) ........................................
[CV] ......................... vect__ngram_range=(1, 1), total= 1.5min
[CV] vect__ngram_range=(1, 1) ........................................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.1min remaining:    0.0s


[CV] ......................... vect__ngram_range=(1, 1), total= 1.5min
[CV] vect__ngram_range=(1, 1) ........................................
[CV] ......................... vect__ngram_range=(1, 1), total= 1.5min
[CV] vect__ngram_range=(1, 2) ........................................
[CV] ......................... vect__ngram_range=(1, 2), total= 2.6min
[CV] vect__ngram_range=(1, 2) ........................................
[CV] ......................... vect__ngram_range=(1, 2), total= 2.6min
[CV] vect__ngram_range=(1, 2) ........................................
[CV] ......................... vect__ngram_range=(1, 2), total= 2.6min


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 16.3min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__ngram_range': [(1, 1), (1, 2)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
# Get the best estimator
model = cv.best_estimator_

# Predict
y_pred = model.predict(X_test)

# Show Performance
show_performance(y_test, y_pred)

Overall Accuracy: 0.94
Overall f-score: 0.6
Overall recall: 0.58
Overall precision: 0.73


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [28]:
# Tested in the same cell.
test = ['thirsty']
# predictions
predict = model.predict(test)

# Get categories
print(y_train.columns.values[(predict.flatten()==1)])

['related' 'water' 'transport']


### 9. Export your model as a pickle file

In [29]:
# Function to save the model as a pickle file
def save_model(model, path):
    file_model = open(path, 'wb')
    pickle.dump(model, file_model)
    file_model.close()

# Load model 
def load_model(path):
    file_model = open(path, 'rb')
    model = pickle.load(file_model)
    return model

save_model(model, 'best_model_cv.pkl')

model = load_model('best_model_cv.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [30]:
test = ['thirsty']
# predictions
predict = model.predict(test)

# Get categories
print(y_train.columns.values[(predict.flatten()==1)])

['related' 'water' 'transport']
